In [111]:
import os
from dotenv import load_dotenv

load_dotenv()  # Reads .env in current directory

CLIENT_ID       = os.getenv("TRAVELPORT_APPLICATION_KEY")
CLIENT_SECRET   = os.getenv("TRAVELPORT_APPLICATION_SECRET")
USERNAME        = os.getenv("TRAVELPORT_USERNAME")
PASSWORD        = os.getenv("TRAVELPORT_PASSWORD")
ACCESS_GROUP    = os.getenv("TRAVELPORT_ACCESS_GROUP")

OAUTH_URL       = "https://oauth.pp.travelport.com/oauth/oauth20/token"
CATALOG_URL     = "https://api.pp.travelport.com/11/air/catalog/search/catalogproductofferings"
AIRPRICE_URL     = "https://api.pp.travelport.com/11/air/price/offers/buildfromcatalogproductofferings"

# CATALOG_URL     = "https://api.pp.travelport.com/11/air/catalog/search/catalogproductofferings"




In [112]:
import requests

def fetch_password_token():
    data = {
        "grant_type":    "password",
        "username":      USERNAME,
        "password":      PASSWORD,
        "client_id":     CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "scope":         "openid"
    }
    resp = requests.post(
        OAUTH_URL,
        headers={"Content-Type": "application/x-www-form-urlencoded"},
        data=data
    )
    resp.raise_for_status()
    return resp.json()["access_token"]

token = fetch_password_token()
print(f"🔑 Token obtained: {token[:20]}…")

🔑 Token obtained: eyJhbGciOiJkaXIiLCJl…


In [113]:
headers = {
    "Accept":                       "application/json",
    "Content-Type":                 "application/json",
    "Accept-Encoding":              "gzip, deflate",
    "Cache-Control":                "no-cache",
    "Authorization":                f"Bearer {token}",
    "XAUTH_TRAVELPORT_ACCESSGROUP": ACCESS_GROUP,
    "Accept-Version":               "11",
    "Content-Version":              "11",
}
print("📋 Headers built")

📋 Headers built


In [114]:
from datetime import datetime, timedelta
import json

# Compute or set your travel dates here:
departure_date = (datetime.utcnow() + timedelta(days=30)).strftime("%Y-%m-%d")
return_date    = (datetime.utcnow() + timedelta(days=37)).strftime("%Y-%m-%d")
from_city = "ATH"
to_city = "SAW"
carrier_list = ["TK"]

# Now build the payload using those real date strings:
payload = {
    "@type": "CatalogProductOfferingsQueryRequest",
    "CatalogProductOfferingsRequest": {
        "@type": "CatalogProductOfferingsRequestAir",
        "maxNumberOfUpsellsToReturn": 1,
        "contentSourceList": ["GDS"],
        "PassengerCriteria": [
            {
                "@type": "PassengerCriteria",
                "number": 1,
                "age": 25,
                "passengerTypeCode": "ADT"
            }
        ],
        "SearchCriteriaFlight": [
            {
                "@type": "SearchCriteriaFlight",
                "departureDate": "2025-08-08",   # injected here
                "From": {"value": from_city},
                "To":   {"value": to_city}
            },
            {
                "@type": "SearchCriteriaFlight",
                "departureDate": return_date,      # and here
                "From": {"value": to_city},
                "To":   {"value": from_city}
            }
        ],
        "SearchModifiersAir": {
            "@type": "SearchModifiersAir",
            "CarrierPreference": [
                {
                    "@type": "CarrierPreference",
                    "preferenceType": "Preferred",
                    "carriers": carrier_list
                }
            ]
        },
        "CustomResponseModifiersAir": {
            "@type": "CustomResponseModifiersAir",
            "SearchRepresentation": "Journey"
        }
    }
}

print(json.dumps(payload, indent=2))


{
  "@type": "CatalogProductOfferingsQueryRequest",
  "CatalogProductOfferingsRequest": {
    "@type": "CatalogProductOfferingsRequestAir",
    "maxNumberOfUpsellsToReturn": 1,
    "contentSourceList": [
      "GDS"
    ],
    "PassengerCriteria": [
      {
        "@type": "PassengerCriteria",
        "number": 1,
        "age": 25,
        "passengerTypeCode": "ADT"
      }
    ],
    "SearchCriteriaFlight": [
      {
        "@type": "SearchCriteriaFlight",
        "departureDate": "2025-08-08",
        "From": {
          "value": "ATH"
        },
        "To": {
          "value": "SAW"
        }
      },
      {
        "@type": "SearchCriteriaFlight",
        "departureDate": "2025-09-03",
        "From": {
          "value": "SAW"
        },
        "To": {
          "value": "ATH"
        }
      }
    ],
    "SearchModifiersAir": {
      "@type": "SearchModifiersAir",
      "CarrierPreference": [
        {
          "@type": "CarrierPreference",
          "preferenceType": "P

In [115]:
response = requests.post(CATALOG_URL, headers=headers, json=payload)
try:
    response.raise_for_status()
    result = response.json()
    import json
    print(json.dumps(result, indent=2))
except requests.HTTPError as e:
    print("❌ Request failed:", e)
    print(response.status_code, response.text)

{
  "CatalogProductOfferingsResponse": {
    "@type": "CatalogProductOfferingsResponse",
    "transactionId": "a016dd27-78f4-4c35-9ff6-160db452a74e",
    "CatalogProductOfferings": {
      "@type": "CatalogProductOfferings",
      "CatalogProductOffering": [
        {
          "@type": "CatalogProductOffering",
          "sequence": 1,
          "id": "o1",
          "Departure": "ATH",
          "Arrival": "IST",
          "Brand": [
            {
              "@type": "BrandID",
              "BrandRef": "b0"
            },
            {
              "@type": "BrandID",
              "BrandRef": "b1"
            },
            {
              "@type": "BrandID",
              "BrandRef": "b2"
            }
          ],
          "ProductBrandOptions": [
            {
              "@type": "ProductBrandOptions",
              "flightRefs": [
                "s2"
              ],
              "ProductBrandOffering": [
                {
                  "@type": "ProductBrandOffer

In [116]:
search_payload = {
  "@type": "CatalogProductOfferingsQueryRequest",
  "CatalogProductOfferingsRequest": {
    "@type": "CatalogProductOfferingsRequestAir",
    "contentSourceList": ["GDS"],
    "maxNumberOfUpsellsToReturn": 1,
    "offersPerPage": 20,
    "PassengerCriteria": [{
      "@type": "PassengerCriteria",
      "passengerTypeCode": "ADT",
      "number": 1,
      "age": 25
    }],
    "SearchCriteriaFlight": [
      { "@type": "SearchCriteriaFlight",
        "departureDate": departure_date,
        "From": {"value": from_city},
        "To":   {"value": to_city}
      },
      { "@type": "SearchCriteriaFlight",
        "departureDate": return_date,
        "From": {"value": to_city},
        "To":   {"value": from_city}
      }
    ],
    "SearchModifiersAir": {
      "@type": "SearchModifiersAir",
      "CarrierPreference": [{
        "@type": "CarrierPreference",
        "preferenceType": "Preferred",
        "carriers": carrier_list
      }]
    },
    "PricingModifiersAir": {
      "@type": "PricingModifiersAir",
      "FareSelection": {"fareType": "PrivateFaresOnly"}
    },
    "CustomResponseModifiersAir": {
      "@type": "CustomResponseModifiersAir",
      "SearchRepresentation": "Journey"
    }
  }
}

resp = requests.post(CATALOG_URL, headers=headers, json=search_payload)
resp.raise_for_status()
result = resp.json()


In [117]:
import requests, json

# … after you do your Search API call …
result = resp.json()

# 1) Peel off the wrapper:
body = result["CatalogProductOfferingsResponse"]

# 2) Grab the list of offerings:
cpo = body.get("CatalogProductOfferings", {})
offerings = cpo.get("CatalogProductOffering") or cpo.get("CatalogProductOffering", [])
# offerings is now either a list or None; coerce to list
if isinstance(offerings, dict):
    offerings = [offerings]
elif offerings is None:
    offerings = []

# 3) Find the cheapest BestCombinablePrice:
cheapest = {"price": float("inf")}

for offer in offerings:
    offer_id = offer["id"]
    for pbo_group in offer.get("ProductBrandOptions", []):
        for pbo in pbo_group.get("ProductBrandOffering", []):
            # parse price
            price_str = pbo["BestCombinablePrice"]["PriceBreakdown"]["total"]
            price = float(price_str)
            if price < cheapest["price"]:
                cheapest = {
                    "price":     price,
                    "offerId":   offer_id,
                    "productRef": pbo["Product"][0]["productRef"],
                    "brandRef":   pbo["Brand"][0]["brandRef"]
                }

print("Cheapest raw search result →", cheapest)


Cheapest raw search result → {'price': inf}


In [128]:
import requests, json
from datetime import datetime, timedelta

# --- 1) Search for private fares (journey-based, cached) ---
departure = (datetime.utcnow() + timedelta(days=30)).strftime("%Y-%m-%d")
return_d  = (datetime.utcnow() + timedelta(days=37)).strftime("%Y-%m-%d")

search_payload = {
  "@type": "CatalogProductOfferingsQueryRequest",
  "CatalogProductOfferingsRequest": {
    "@type": "CatalogProductOfferingsRequestAir",
    "contentSourceList": ["GDS"],
    "maxNumberOfUpsellsToReturn": 1,
    "offersPerPage": 20,
    "PassengerCriteria":[{"@type":"PassengerCriteria","passengerTypeCode":"ADT","number":1,"age":25}],
    "SearchCriteriaFlight":[
      {"@type":"SearchCriteriaFlight","departureDate":"2025-09-15","From":{"value":"IST"},"To":{"value":"ATH"}},
      # {"@type":"SearchCriteriaFlight","departureDate":"2025-09-15",  "From":{"value":"IST"},"To":{"value":"ATH"}}
    ],
    "SearchModifiersAir":{"@type":"SearchModifiersAir","CarrierPreference":[{"@type":"CarrierPreference","preferenceType":"Preferred","carriers":["TK"]}]},
    "PricingModifiersAir":{"@type":"PricingModifiersAir","FareSelection":{"@type":"FareSelection","fareType":"AllFares"}},
    "CustomResponseModifiersAir":{"@type":"CustomResponseModifiersAir","SearchRepresentation":"Journey"}
  }
}

resp = requests.post(CATALOG_URL, headers=headers, json=search_payload)
resp.raise_for_status()
body = resp.json()["CatalogProductOfferingsResponse"]

# --- 2) Find cheapest offer/product ---
offers_block = body["CatalogProductOfferings"]
session_id   = offers_block["Identifier"]["value"]
offerings    = offers_block["CatalogProductOffering"]
if isinstance(offerings, dict):
    offerings = [offerings]

cheapest = {"price": float("inf")}
for o in offerings:
    oid = o["id"]
    for grp in o.get("ProductBrandOptions", []):
        for pbo in grp.get("ProductBrandOffering", []):
            bcp = pbo["BestCombinablePrice"]
            tp = bcp.get("TotalPrice") or bcp.get("PriceBreakdown", [{}])[0].get("Amount", {}).get("Total")
            if tp is None:
                continue
            price = float(tp)
            if price < cheapest["price"]:
                cheapest = {
                    "price":      price,
                    "offerId":    oid,
                    "productRef": pbo["Product"][0]["productRef"]
                }

print("🔍 Cheapest raw search result →", cheapest)

# --- 3) Build & send AirPrice Reference payload ---
reference_payload = {
  "OfferQueryBuildFromCatalogProductOfferings": {
    "@type": "OfferQueryBuildFromCatalogProductOfferings",
    "BuildFromCatalogProductOfferingsRequestAir": {
      "@type": "BuildFromCatalogProductOfferingsRequestAir",
      "CatalogProductOfferingsIdentifier": {
        "Identifier": { "value": session_id }
      },
      "CatalogProductOfferingSelection": [
        {
          "CatalogProductOfferingIdentifier": {
            "Identifier": { "value": cheapest["offerId"] }
          },
          "ProductIdentifier": [
            { "Identifier": { "value": cheapest["productRef"] } }
          ]
        }
      ]
    }
  }
}


apr = requests.post(AIRPRICE_URL, headers=headers, json=reference_payload)
if not apr.ok:
    print(f"AirPrice Reference failed [{apr.status_code}]:\n{apr.text}")
    raise SystemExit

print("✅ AirPrice Reference response:\n", json.dumps(apr.json(), indent=2))


🔍 Cheapest raw search result → {'price': 123.92, 'offerId': 'o1', 'productRef': 'p0'}
✅ AirPrice Reference response:
 {
  "OfferListResponse": {
    "@type": "OfferListResponse",
    "transactionId": "5f268a27-b5c8-4a13-880f-fe7309e64ccc:ppn",
    "OfferID": [
      {
        "@type": "Offer",
        "id": "o0",
        "ContentSource": "GDS",
        "Product": [
          {
            "@type": "ProductAir",
            "totalDuration": "PT1H25M",
            "id": "p0",
            "FlightSegment": [
              {
                "@type": "FlightSegment",
                "id": "s1",
                "sequence": 1,
                "Flight": {
                  "@type": "FlightDetail",
                  "distance": 343,
                  "duration": "PT1H25M",
                  "carrier": "TK",
                  "number": "1841",
                  "operatingCarrier": "TK",
                  "operatingCarrierName": "Turkish Airlines Inc",
                  "equipment": "32B",
       